In [1]:
from utils import *
from api_calls import *
import pandas as pd
from time import sleep

In [298]:
gt_closed_channels = read_json(results_folder + 'closedchannels.json')['channels']

In [473]:
# # read old data
# funding_txs = read_json(level1_folder + 'funding_txs.json')
# ###############
# channels = pd.read_csv(level2_folder + 'channel.csv')
# nodes = pd.read_csv(level2_folder + 'node.csv')
# ip_addresses = pd.read_csv(level2_folder + 'ip_address.csv')

In [474]:
# read new data
## friedhelm
channels_friedhelm = pd.read_csv('../data/friedhelm/channels020120-090920.csv').drop(columns=['first_seen'])
nodes_friedhelm = pd.read_csv('../data/friedhelm/node020120-090920.csv')
ip_addresses_friedhelm = pd.read_csv('../data/friedhelm/ip_address020120-090920.csv')

In [ ]:
# read new data
## peter
channels_peter = pd.read_csv('../data/peter/for_matteo/channels.csv')
nodes_peter = pd.read_csv('../data/peter/for_matteo/nodes.csv')
# read gt data
gt_channels = pd.read_csv('../data/joined/level_2/ground_truth_channel.csv')

# merge ip addresses

In [475]:
ip_addresses.shape

(5493, 2)

In [476]:
ip_addresses_friedhelm.shape

(10278, 2)

In [477]:
node_ips = dict()
for r in ip_addresses.values:
    node, ips = r
    ips = set(ips.split('|'))
    if node not in node_ips:
        node_ips[node] = set()
    node_ips[node] = node_ips[node].union(ips)

In [478]:
for r in ip_addresses_friedhelm.values:
    node, ips = r
    ips = set(ips.split('|'))
    if node not in node_ips:
        node_ips[node] = set()
    node_ips[node] = node_ips[node].union(ips)

In [479]:
len(node_ips)

7502

In [480]:
for n, ip in node_ips.items():
    node_ips[n] = '|'.join(ip)

In [481]:
node_ips_df = pd.DataFrame(node_ips.items(), columns=['pub_key', 'ip_address'])

In [505]:
node_ips_df.to_csv(level2_folder + 'ip_address.csv', index=False)

# merge aliases

In [484]:
nodes.shape

(6501, 2)

In [485]:
nodes_friedhelm.shape

(11785, 2)

In [495]:
node_aliases = dict()
for r in nodes.values:
    node, aliases = r
    aliases = set(aliases.split('|'))
    if node not in node_aliases:
        node_aliases[node] = set()
    node_aliases[node] = node_aliases[node].union(aliases)

In [488]:
for r in nodes_friedhelm.values:
    node, aliases = r
    if isinstance(aliases, str):
        aliases = set(aliases.split('|'))
        if node not in node_aliases:
            node_aliases[node] = set()
        node_aliases[node] = node_aliases[node].union(aliases)

In [496]:
for r in nodes_peter.values:
    node, aliases = r
    if isinstance(aliases, str):
        aliases = set(aliases.split('|'))
        if node not in node_aliases:
            node_aliases[node] = set()
        node_aliases[node] = node_aliases[node].union(aliases)

In [497]:
len(node_aliases)

8920

In [498]:
for n, a in node_aliases.items():
    node_aliases[n] = '|'.join(a)

In [499]:
node_aliases_df = pd.DataFrame(node_aliases.items(), columns=['pub_key', 'alias'])

In [504]:
node_aliases_df.to_csv(level2_folder + 'node.csv', index=False)

In [336]:
# local_node = '025228840b37ade9aa2f96b3c961a35e76571a7c87a4ee67e2f33c64de64aa822f'
# print('chan_point,node1,node2')
# for ch in gt_closed_channels:
#     if ch['channel_point'] not in merged_channel_points:
#         print(ch['channel_point'] + ',' + ch['remote_pubkey'] + ',' + local_node)
# print(inbound_channels.values[-1][0] + ',' + inbound_channels.values[-1][1] + ',' + local_node)

# merge channels

In [187]:
channels.shape

(70783, 3)

In [188]:
channels_friedhelm.shape

(62989, 3)

In [189]:
channels_peter.shape

(44902, 3)

In [197]:
merged_channels = channels.merge(channels_friedhelm, how='outer', on='chan_point')

In [198]:
merged_channels.shape

(93602, 5)

In [199]:
merged_channels['node1'] = merged_channels['node1_pub_x'].where(merged_channels['node1_pub_x'].notnull(), merged_channels['node1_pub_y'])
merged_channels['node2'] = merged_channels['node2_pub_x'].where(merged_channels['node2_pub_x'].notnull(), merged_channels['node2_pub_y'])
merged_channels = merged_channels[['chan_point', 'node1', 'node2']]

In [200]:
merged_channels = merged_channels.merge(channels_peter, how='outer', on='chan_point')

In [201]:
merged_channels.shape

(99270, 5)

In [204]:
merged_channels['node1'] = merged_channels['node1'].where(merged_channels['node1'].notnull(), merged_channels['node1_pub'])
merged_channels['node2'] = merged_channels['node2'].where(merged_channels['node2'].notnull(), merged_channels['node2_pub'])
merged_channels = merged_channels[['chan_point', 'node1', 'node2']]


In [660]:
merged_channels.shape

(99295, 3)

In [341]:
merged_channels = merged_channels.merge(gt_channels, how='outer', on='chan_point')

In [661]:
merged_channels.shape

(99295, 3)

In [343]:
merged_channels['node1'] = merged_channels['node1_x'].where(merged_channels['node1_x'].notnull(), merged_channels['node1_y'])
merged_channels['node2'] = merged_channels['node2_x'].where(merged_channels['node2_x'].notnull(), merged_channels['node2_y'])
merged_channels = merged_channels[['chan_point', 'node1', 'node2']]


In [672]:
merged_channels.shape

(99295, 4)

In [682]:
merged_channels['tx'] = merged_channels.apply(lambda x: x[0].split(':')[0], axis=1)

/home/matteo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [690]:
merged_channels = merged_channels.loc[~merged_channels['tx'].isin(not_available_ftxs)]

In [692]:
merged_channels = merged_channels[['chan_point', 'node1', 'node2']]

In [693]:
merged_channels.to_csv(level2_folder + 'channel.csv', index=False)

# Funding txs

In [647]:
last_block = 647529

In [527]:
for channel in merged_channels.chan_point.values:
    hsh = channel.split(':')[0]
    if hsh not in funding_txs:
        funding_txs[hsh] = None

In [709]:
for i, hsh in enumerate(funding_txs):
    # value must be dict and with details
    if not isinstance(funding_txs[hsh], dict):
        print('Fetching funding tx', i, end='\r')
        funding_txs[hsh] = get_blockstream_tx(hsh)

In [650]:
not_available_ftxs = []
for hsh, ftx in funding_txs.items():
    block = ftx['status']['block_height']
    if block > last_block:
        not_available_ftxs.append(hsh)

In [698]:
available_funding_txs = dict()
for hsh, tx in funding_txs.items():
    if hsh not in not_available_ftxs:
        available_funding_txs[hsh] = tx

In [700]:
write_json(available_funding_txs, level1_folder + 'blockstream_funding_txs.json')

In [713]:
funding_txs = read_json(level1_folder + 'blockstream_funding_txs.json')

In [716]:
len(funding_txs)

98240

# funded address settlement txs

In [ ]:
for i, channel in enumerate(merged_channels.chan_point.values):
    hsh, out_index = channel.split(':')
    funded_address = funding_txs[hsh]['vout'][int(out_index)]['scriptpubkey_address']        
    if funded_address not in funded_address_settlement_txs:
        funded_address_settlement_txs[funded_address] = None

In [ ]:
max_reached = 0

In [ ]:
for i, funded_address in enumerate(funded_address_settlement_txs):
    if i >= max_reached:
        if not isinstance(funded_address_settlement_txs[funded_address], list) or not funded_address_settlement_txs[funded_address]:
            print(i, end='\r')
            txs = get_blockstream_address_txs(funded_address)
            if len(txs) > 2:
                # doesn't happen in our dataset
                print('Address with > 2 settlement txs:', len(txs), funded_address)
            for tx in txs:
                if funded_address in [e['prevout']['scriptpubkey_address'] for e in tx['vin']]:
                    # funded address is in inputs 
                    funded_address_settlement_txs[funded_address] = []
                    funded_address_settlement_txs[funded_address].append(tx)
        max_reached += 1

In [794]:
write_json(funded_address_settlement_txs, level1_folder + 'funded_address_blockstream_settlement_txs.json')

In [795]:
funded_address_settlement_txs = read_json(level1_folder + 'funded_address_blockstream_settlement_txs.json')

In [772]:
len(funded_address_settlement_txs)

98431

In [780]:
settlement_txs = dict()
for stxs in funded_address_settlement_txs.values():
    for stx in stxs:
        settlement_txs[stx['txid']] = stx

In [813]:
settlement_txs[stx['txid']]

{'txid': '95d8f9b1e0e34ad12a022d0d0871bb59f8e3c0db20e89d055da40a1b25cc5e9d',
 'version': 2,
 'locktime': 0,
 'vin': [{'txid': 'e7b2d73d2f7e41f00e0818d483e189ea7097a98b30e39fc7328d0af80d1c8fc8',
   'vout': 1,
   'prevout': {'scriptpubkey': '002035cf0dca0bf5a1470cb3357ad40cae42566e028e1ca7ee4fad1b66d28fef5301',
    'scriptpubkey_asm': 'OP_0 OP_PUSHBYTES_32 35cf0dca0bf5a1470cb3357ad40cae42566e028e1ca7ee4fad1b66d28fef5301',
    'scriptpubkey_type': 'v0_p2wsh',
    'scriptpubkey_address': 'bc1qxh8smjst7ks5wr9nx4adgr9wgftxuq5wrjn7unadrdnd9rl02vqswxu7fa',
    'value': 100000},
   'scriptsig': '',
   'scriptsig_asm': '',
   'witness': ['',
    '304402202f5ddf2fd2cafef630533ffb640c5614b5c2ac910220f7aa5661448ea2473b1d02204889d0727301f6e330d3cade82f5d121faafe5472027dce1c415c3817cd453ef01',
    '3045022100dd224d458340db57e7e10ef16e7c57eb29e8fee99a05ce1c2166c8b5217555c102207dee634910cba52cca178ff93017e676de3238dd21a20b64b9ef40cf43cd6dde01',
    '5221030ccbb5b5d57bf8f451007c654e47395dc90ad3492bc9a6d

In [781]:
len(settlement_txs)

60447

In [810]:
write_json(settlement_txs, level1_folder + 'blockstream_settlement_txs.json')

In [784]:
# highest_block_stxs = 0
# for stx in settlement_txs.values():
#     if stx['status']['block_height'] > highest_block_stxs:
#         highest_block_stxs = stx['status']['block_height']
# highest_block_stxs

In [773]:
len([stxs for stxs in funded_address_settlement_txs.values() if stxs])

60447

In [767]:
# available_funded_address_settlement_txs = dict()
# for fa, stxs in funded_address_settlement_txs.items():
#     available_funded_address_settlement_txs[fa] = []
#     for stx in stxs:
#         if 'block_height' in stx['status'] and not stx['status']['block_height'] > last_block:
#             available_funded_address_settlement_txs[fa].append(stx)

In [ ]:
# write_json(available_funded_address_settlement_txs, level1_folder + 'funded_address_blockstream_settlement_txs.json')

In [ ]:
settlement_txs_with_punishment = read_json(results_folder + 'settlement_txs_with_punishment.json')

In [792]:
# non_punishment_funded_address_settlement_txs = dict()
# for fa, stxs in funded_address_settlement_txs.items():
#     non_punishment_funded_address_settlement_txs[fa] = []
#     for stx in stxs:
#         if stx['txid'] not in settlement_txs_with_punishment:
#             non_punishment_funded_address_settlement_txs[fa].append(stx)

In [796]:
# len([stxs for stxs in non_punishment_funded_address_settlement_txs.values() if stxs])

In [ ]:
# write_json(non_punishment_funded_address_settlement_txs, level1_folder + 'funded_address_blockstream_settlement_txs.json')

In [ ]:
settlement_txs_dict = 

# funding addresses entities

In [731]:
funding_addresses = set()
for hsh, tx in funding_txs.items():
    for vin in tx['vin']:
        funding_addresses.add(vin['prevout']['scriptpubkey_address'])
funding_addresses = list(funding_addresses)

In [732]:
len(funding_addresses)

169141

In [734]:
write_json(funding_addresses, level1_folder + 'funding_addresses.json')

In [785]:
funding_address_entity_df = pd.read_csv(level1_folder + 'funding_addresses_entities.csv')

In [786]:
len(funding_address_entity_df)

170777

In [743]:
funding_address_entity = dict()
funding_addresses_set = set(funding_addresses)
for el in funding_address_entity_df.values:
    a, e = el
    try:
        e = int(e)
        if a in funding_addresses_set:
            funding_address_entity[a] = e
    except:
        pass

In [809]:
write_json(funding_address_entity, level1_folder + 'funding_address_entity.json')

In [746]:
len([1 for e in funding_address_entity.values() if isinstance(e, int)])

169141

# settlement addresses entities

In [804]:
settlement_addresses = set()
for fa, stxs in funded_address_settlement_txs.items():
    for tx in stxs:
        for vout in tx['vout']:
            settlement_addresses.add(vout['scriptpubkey_address'])

In [798]:
len(settlement_addresses)

88165

In [808]:
write_json(list(settlement_addresses), level1_folder + 'settlement_addresses.json')

In [799]:
settlement_address_entity_df = pd.read_csv(level1_folder + 'settlement_addresses_entities.csv')

In [800]:
len(settlement_address_entity_df)

88166

In [802]:
settlement_address_entity = dict()
for el in settlement_address_entity_df.values:
    a, e = el
    e = int(e)
    if a in settlement_addresses:
        settlement_address_entity[a] = e
    else:
        print(a) # why are you here?

bc1qutfquvfv7hwekuc69vwsm2s8vt4pefakyruf8l


In [805]:
len([1 for e in settlement_address_entity.values()])

88165

In [806]:
write_json(settlement_address_entity, level1_folder + 'settlement_address_entity.json')

In [596]:
write_json(blockstream_settlement_txs, level1_folder + 'blockstream_settlement_txs.json')

In [513]:
settlement_txs_to_exclude = read_json(results_folder + 'settlement_txs_with_punishment.json')

In [811]:
len(funding_txs)

98240

In [812]:
len(funded_address_settlement_txs)

98431

# old

In [46]:
# fails = 1
# while fails:
#     fails = 0
#     for i, funded_address in enumerate(funded_address_settlement_txs):
#         if not isinstance(funded_address_settlement_txs[funded_address], list) or not funded_address_settlement_txs[funded_address]:
#             try:
#                 print('Fetching txs of address', i, end='\r')
#                 txs = get_address_txs(funded_address)
#                 if len(txs) > 2:
#                     # doesn't happen in our dataset
#                     print('Address with > 2 settlement txs:', len(txs), funded_address)
#                 for tx in txs:
#                     if tx['value']['value'] < 0: # spending coins
#                         d = get_tx(tx['tx_hash'])
#                         if 'message' not in d:
#                             # not an error
#                             funded_address_settlement_txs[funded_address] = []
#                             funded_address_settlement_txs[funded_address].append(d)
#             except Exception as e:
#                 fails += 1
#                 print(e)


In [138]:
def get_blockstream_address_txs(address):
    url = 'https://blockstream.info/api/address/' + address + '/txs'
    try:
        d = url2dict(url)
        return d
    except Exception as e:
        print(e)

In [355]:
missing = 0
for i, funded_address in enumerate(funded_address_settlement_txs):
    if not isinstance(funded_address_settlement_txs[funded_address], list) or not funded_address_settlement_txs[funded_address]:
        missing += 1
        
missing

38130

In [233]:
for i, funded_address in enumerate(funded_address_settlement_txs):
    if isinstance(funded_address_settlement_txs[funded_address], list) and len(funded_address_settlement_txs[funded_address]) == 0:
        print(i, end='\r')
        txs = get_blockstream_address_txs(funded_address)
        if len(txs) > 2:
            # doesn't happen in our dataset
            print('Address with > 2 settlement txs:', len(txs), funded_address)
        for tx in txs:
            if funded_address in [e['prevout']['scriptpubkey_address'] for e in tx['vin']]:
                # funded address is in inputs 
                funded_address_settlement_txs[funded_address] = []
                funded_address_settlement_txs[funded_address].append(tx)

In [359]:
for i, funded_address in enumerate(funded_address_settlement_txs):
    if not isinstance(funded_address_settlement_txs[funded_address], list):
        print(i, end='\r')
        txs = get_blockstream_address_txs(funded_address)
        if len(txs) > 2:
            # doesn't happen in our dataset
            print('Address with > 2 settlement txs:', len(txs), funded_address)
        for tx in txs:
            if funded_address in [e['prevout']['scriptpubkey_address'] for e in tx['vin']]:
                # funded address is in inputs 
                funded_address_settlement_txs[funded_address] = []
                funded_address_settlement_txs[funded_address].append(tx)


In [514]:
len([stxs for stxs in funded_address_settlement_txs.values() if stxs]) # n closed channels

61189

In [362]:
for i, funded_address in enumerate(funded_address_settlement_txs):
    # either empty list or None
    if isinstance(funded_address_settlement_txs[funded_address], list) and len(funded_address_settlement_txs[funded_address]) == 0:
        funded_address_settlement_txs[funded_address] = []
    if not funded_address_settlement_txs[funded_address]:
        funded_address_settlement_txs[funded_address] = []

In [515]:
write_json(funded_address_settlement_txs, level1_folder + 'funded_address_settlement_txs.json')

In [588]:
gs = 0
bs = 0
settlement_txs_dict = dict()
for stxs in funded_address_settlement_txs.values():
    for stx in stxs:
        if 'tx_hash' in stx:
            gs += 1
            settlement_txs_dict[stx['tx_hash']] = stx
        else:
            bs += 1
            settlement_txs_dict[stx['txid']] = stx
print(gs, bs)

33503 27686


In [587]:
len([el for el in settlement_txs_dict.values() if 'txid' in el])

27686

In [512]:
write_json(settlement_txs_dict, level1_folder + 'settlement_txs_dict.json')

In [548]:
funding_addresses = set()
for hsh, tx in funding_txs.items():
    if 'vin' in tx:
        for vin in tx['vin']:
            funding_addresses.add(vin['prevout']['scriptpubkey_address'])
    elif 'inputs' in tx:
        for vin in tx['inputs']:
            funding_addresses.add(vin['address'])
    else:
        print(tx)

In [549]:
len(funding_addresses)

170777

In [517]:
settlement_addresses = set()
for fa, stxs in funded_address_settlement_txs.items():
    for tx in stxs:
        if 'vout' in tx:
            if tx['txid'] not in settlement_txs_to_exclude:
                for vout in tx['vout']:
                    settlement_addresses.add(vout['scriptpubkey_address'])
        elif 'outputs' in tx:
            if tx['tx_hash'] not in settlement_txs_to_exclude:
                for vout in tx['outputs']:
                    settlement_addresses.add(vout['address'])
        else:
            print(tx)

In [367]:
len(funded_address_settlement_txs)

99295

In [543]:
len(settlement_addresses)

89337

In [541]:
funding_addresses_df = pd.DataFrame(funding_addresses, columns=['address'])

In [544]:
settlement_addresses_df = pd.DataFrame(settlement_addresses, columns=['address'])

In [542]:
funding_addresses_df.to_csv(level1_folder + 'funding_addresses.csv', index=False)

In [545]:
settlement_addresses_df.to_csv(level1_folder + 'settlement_addresses.csv', index=False)

In [285]:
blockstream_funding_txs = read_json(level1_folder + 'blockstream_funding_txs_all.json')

In [286]:
len(blockstream_funding_txs)

5550

In [290]:
len(funding_txs)

99055

In [289]:
for i, hsh in enumerate(funding_txs):
    if hsh not in blockstream_funding_txs:
        if 'vin' not in funding_txs[hsh]:
            blockstream_funding_txs[hsh] = get_blockstream_tx(hsh)
            print(i, end='\r')
        else:
            blockstream_funding_txs[hsh] = funding_txs[hsh]

In [291]:
write_json(blockstream_funding_txs, level1_folder + 'blockstream_funding_txs.json')

In [292]:
wasabi_addresses = ['bc1qs604c7jv6amk4cxqlnvuxv26hv3e48cds4m0ew', 'bc1qa24tsgchvuxsaccp8vrnkfd85hrcpafg20kmjw']

In [293]:
output_addresses_funding_txs = set()
for hsh, tx in funding_txs.items():
    if 'vout' in tx:
        for vout in tx['vout']:
            output_addresses_funding_txs.add(vout['scriptpubkey_address'])
    elif 'outputs' in tx:
        for vout in tx['outputs']:
            output_addresses_funding_txs.add(vout['address'])
    else:
        print(tx)

In [295]:
len(funding_addresses)

170728

In [294]:
len(output_addresses_funding_txs)

194616